In [1]:
import chess
import chess.pgn
import chess.engine
import requests
import io
import json
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [2]:
%run ../bedrock_setup.py

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings_model = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

/home/ec2-user/SageMaker/spring-2025-final-project-project-group-13/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
from langchain.chat_models import init_chat_model

model = init_chat_model("us.anthropic.claude-3-5-haiku-20241022-v1:0",
                        model_provider="bedrock_converse",
                        region_name="us-east-1",
                        client=bedrock_client)

In [18]:
hikaru_vs = FAISS.load_local("../database/Hikaru", embeddings_model, allow_dangerous_deserialization=True)
# carlsen_vs = FAISS.load_local("../database/MagnusCarlsen", embeddings_model, allow_dangerous_deserialization=True)
# fabiano_vs = FAISS.load_local("../database/FabianoCaruana", embeddings_model, allow_dangerous_deserialization=True)
# arjun_vs = FAISS.load_local("../database/GHANDEEVAM2003", embeddings_model, allow_dangerous_deserialization=True)
# gukesh_vs = FAISS.load_local("../database/GukeshDommaraju", embeddings_model, allow_dangerous_deserialization=True)
all_games = FAISS.load_local("../database/all_games", embeddings_model, allow_dangerous_deserialization=True)

In [31]:
# fen = "1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3"
fen = "rnbq1rk1/ppp1bppp/3p1n2/8/2BP4/2N2N2/PPP2PPP/R1BQ1RK1 b - - 4 7"

In [37]:
len(all_games.similarity_search(fen, k=10, filter={"black": "Hikaru"}))

# results = all_games.similarity_search(fen, k=100)
# len([doc for doc in results if doc.metadata.get("black") == "Hikaru"][:10])

8

In [33]:
hikaru_vs.similarity_search(fen, k=10)

[Document(id='3f0d14da-e179-497b-b917-349e7c2be358', metadata={'date': '2018.10.12', 'white': 'DanielNaroditsky', 'whiteElo': '2976', 'black': 'Hikaru', 'blackElo': '3017', 'result': '0-1', 'termination': 'Hikaru won by resignation', 'url': 'https://www.chess.com/game/live/3138388410', 'move': 'Nc6', 'fen': 'rnbq1rk1/ppp2pbp/3p1np1/4p3/2PPP3/2N2N2/PP2BPPP/R1BQ1RK1 b - - 1 7'}, page_content='rnbq1rk1/ppp2pbp/3p1np1/4p3/2PPP3/2N2N2/PP2BPPP/R1BQ1RK1 b - - 1 7'),
 Document(id='2a42ff3b-bcee-46e6-a016-30f8e6a44090', metadata={'date': '2015.11.21', 'white': 'Noukii', 'whiteElo': '2669', 'black': 'Hikaru', 'blackElo': '3074', 'result': '0-1', 'termination': 'Hikaru won by resignation', 'url': 'https://www.chess.com/game/live/1358368220', 'move': 'Nc6', 'fen': 'rnbq1rk1/ppp2pbp/3p1np1/4p3/2PPP3/2N2N2/PP2BPPP/R1BQ1RK1 b - - 1 7'}, page_content='rnbq1rk1/ppp2pbp/3p1np1/4p3/2PPP3/2N2N2/PP2BPPP/R1BQ1RK1 b - - 1 7'),
 Document(id='3425c35f-2a23-4c62-a74a-54925154aa39', metadata={'date': '2024.07.05

In [11]:
engine = chess.engine.SimpleEngine.popen_uci("../Stockfish/src/stockfish")

In [59]:
@tool
def extract_fen(query: str) -> str:
    """Extract FEN notation from the user query"""
    import re
    # Look for a pattern that resembles FEN notation
    fen_pattern = r'[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+/[rnbqkpRNBQKP1-8]+ [wb] [KQkq-]+ [a-h1-8-]+ \d+ \d+'
    match = re.search(fen_pattern, query)
    if match:
        return match.group(0)
    return "No FEN notation found in query"

@tool
def is_gradmaster_specific(query: str) -> str:
    """Get grandmaster name from the query"""
    import re
    pattern = r'carlsen|hikaru|gukesh|arjun|fabiano'
    match = re.search(pattern, str.lower(query))
    if match:
        return match.group(0)
    else:
        return "None"

@tool
def query_knowledge_base(name:str, fen: str) -> set:
    """Get relevant documents on a relevant vectorstore based on FEN from the query"""
    if name != "None":
        docs = all_games.similarity_search(fen, k=10)
        
    elif name == 'carlsen':
        docs = carlsen_vs.similarity_search(fen, k=10)
    elif name == 'hikaru':
        docs = hikaru_vs.similarity_search(fen, k=10)
    elif name == 'fabiano':
        docs = fabiano_vs.similarity_search(fen, k=10)
    elif name == 'gukesh':
        docs = gukesh_vs.similarity_search(fen, k=10)
    elif name == 'arjun':
        docs = arjun_vs.similarity_search(fen, k=10)
        
    return list(set([doc.metadata['move'] for doc in docs]))

@tool
def evaluate_move(fen: str, move:str) -> int:
    """
    Calculate a centipawn score using Stockfish
    """
    board = chess.Board(fen)    
    board.push_san(move)
    score = engine.analyse(board, chess.engine.Limit(depth=15))["score"].relative.score()
    return score
    
@tool
def recommend_move(fen:str, content:str, moves: list, scores:list) -> str:
    """
    Recommend a move based on the list of PGN of chess games and FEN
    """
    recommened_move = model.invoke(f"Recommend a move based on the FEN using content from the vectorstore")
    return recommend_move
    
@tool
def validate_move(fen:str, move:str) -> bool:
    """
    Validate whether the recommended move is legal or not on a current chess board state.
    """
    board = chess.Board(fen)
    try:
        board.push_san(move)
        return True
    except:
        return False

In [34]:
fen = "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R B KQkq - 2 3?"
docs = hikaru_vs.similarity_search(fen, k=10)

best_score = 0
for doc in docs:
    board = chess.Board(doc.metadata["fen"])
    board.push_san(doc.metadata["move"])
    
    score = engine.analyse(board, chess.engine.Limit(depth=15))["score"].relative.score()
    print(doc.metadata["move"])
    print(score)
    
    if score < best_score:
        best_score = score
        best_move = doc.metadata["move"]
        
print(best_move)

Nf6
-132
Bc5
15
Bc5
14
Bc5
16
Nf6
17
Bc5
17
Nf6
15
Bc5
13
Bc5
16
f6
171
Nf6


In [60]:
tools = [extract_fen, is_gradmaster_specific, query_knowledge_base, evaluate_move, recommend_move, validate_move]

In [61]:
prompt = """
You are an intelligent agent that recommend chess moves to users.
Follow these steps to answer user queries:
1. Use the `extract_fen` tool to extract FEN from the user query.
2. Use the `is_gradmaster_specific` tool to decide whether the user query is gradmaster specific or not.
3. Use the `query_knowledge_base` tool to retrieve relevant documents from proper vectorstores.
4. Use the `evaluate_move` tool to calculate the centipawn score of the move.
5. Use the `recommend_move` tool to recommend moves based on the FEN using relevant documents.
6. Use the 'validate_move' tool to make sure the recommended move is valid on the current board state.
7. If you can't find enough information, state that explicitly.

Use a step-by-step approach to complete each query.
"""

In [62]:
agent_executor = create_react_agent(model, tools, prompt = prompt)

In [65]:
query = "What would I do in this position: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3?"
response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})

In [66]:
response

{'messages': [HumanMessage(content='What would I do in this position: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3?', additional_kwargs={}, response_metadata={}, id='e18dd3fe-005f-44b3-b582-5c8b74caaad7'),
  AIMessage(content=[{'type': 'text', 'text': "I'll help you analyze this chess position step by step:\n\n1. First, I'll extract the FEN notation (which is already provided in the query):\nr1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3\n\n2. Let's check if this is a grandmaster-specific query:"}, {'type': 'tool_use', 'name': 'is_gradmaster_specific', 'input': {'query': 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3'}, 'id': 'tooluse__oCoXTczQ3mxQ2zc8Lfo9Q'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'c6f0f554-41ec-4f46-bafd-d185b85b8129', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 02:31:43 GMT', 'content-type': 'application/json', 'content-length': '737', 'connection': 'ke

In [67]:
response["messages"][-1].content

'Based on the analysis, I recommend playing Nc3. This move:\n1. Develops another piece (the knight)\n2. Controls the center\n3. Prepares for potential kingside castling\n4. Has the best evaluation score among the considered moves\n\nThe position looks like an early stage of the Ruy Lopez or Spanish Opening, where White is developing pieces and preparing to contest the center. The move Nc3 supports the e4 pawn and adds pressure to the center of the board.\n\nThe move is perfectly legal and follows good opening principles of piece development and central control.'

In [68]:
fen = 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3'

board = chess.Board(fen)

move_san = "Nc3"  # Example move

# Check if the move is legal
try:
    # Try pushing the move
    move = board.push_san(move_san)
    print(f"Move {move_san} is valid.")
except ValueError as e:
    print(f"Move {move_san} is not valid: {e}")

Move Nc3 is valid.
